In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import xlwings as xw

wb = xw.Book('c:\\Users\\moholom\\Desktop\\NBA\\Excel\\LeBron_Inputs.xlsm')
sht = wb.sheets['LeBron_Outputs']

LeBron = pd.read_excel('c:\\Users\\moholom\\Desktop\\NBA\\Excel\\LeBron_Inputs.xlsm')

LeBron_Pts_per_Min = LeBron['Points/Min_Infor'][0]
#Taken from my Projections Sheet vs League Average (as Usage Rank 1)

pts_per_min_st_dev = LeBron['Points/Min _StDev'][0]
#Found through some quick research

proj_games = int(LeBron['Proj._Games'][0])
#Can easily change this

games_st_dev = LeBron['Proj. Games_StDev'][0]
#This is aribtrary for now but can be made more precise

runs = int(LeBron['No. of Sims'][0])
#N number of runs

total = []
games_played = []
for i in range(runs +1): #simulate n number of times

    current = int(LeBron['Current_Tally'][0])
    #LeBron's current Points Total
    games = []
    missed = []

    played = int(np.random.normal(proj_games, games_st_dev)) - int(LeBron['Games_Played'][0])
    #Random game total for season from normal distribution

    if played > 82:
        played = 82
    elif played < 0:
        played = 0

    games = [1] * (played) 
    #Creates a list filled with 1s, the length of which is equal to the simulated game total in this iteration

    missed = [0] * (82 - played) 
    #Creates another list filled with 0s, the length of which is 82 - games played

    games.extend(missed) 
    random.shuffle(games) 
    #This merges the lists and shuffles them, now a list 82 characters long, 1 for game played, 0 for game missed 

   
    for g in range(0, len(games)):
        #can only score in games he plays, represented by 1s

        if games[g] == 0:
            continue

        elif games[g] == 1:
            scored_per_min = np.random.normal(LeBron_Pts_per_Min, (pts_per_min_st_dev)) 
            #simulate scoring output per min 

            min_played = np.random.normal(32, 5) 
            #simulate minutes played around 32 as average, can change

            if min_played > 48:
                min_played = 48
            elif min_played < 0:
                min_played = 0 
                #only observe match-meaningful minute totals

            scored = int(scored_per_min * min_played)
            #game output as an integer

            current += scored 
            #add to LeBron total

        if current > 40000: 
            #stop the season simulation if 40k is reached
            games_taken = g + 1
            break 

    total.append(games_taken)
    games_played.append(played)

sns.displot(total, stat='probability')
plt.show()

proportion = []

for t in list(np.unique(total)):
    prop = total.count(t)/len(total)
    proportion.append(prop)

games = pd.DataFrame(list(np.unique(total)), columns= ['Games'])

proportion = pd.DataFrame(proportion, columns = ['Probability'])
proportion

games['Probability'] = proportion['Probability']
games.set_index('Games', inplace=True)
games['Price (€)'] = round(1 / games['Probability'], 2)

sht.range('A1:C1').value = games
sht.range('F2').value = np.mean(total)
sht.range('H2').value = np.mean(games_played)
wb.save('c:\\Users\\moholom\\Desktop\\NBA\\Excel\\LeBron_Inputs.xlsm')